# Semantic Kernel : Chat, Planners, Memory, Hugging Face, Groundedness & Multi-Result

Dans ce notebook, nous allons explorer plusieurs fonctionnalités avancées de **Semantic Kernel** :

1. **Chat basique** avec Kernel Arguments (historique et contexte via un objet `KernelArguments`).
2. **Planners** (Sequential, Stepwise Function Calling) pour orchestrer dynamiquement des actions selon un but donné.
3. **Mémoire** & Embeddings (VolatileMemoryStore ou connecteurs externes) pour stocker des informations sémantiques.
4. **Hugging Face** : Intégration de modèles (texte, embeddings) en local ou depuis le Hub.
5. **Groundedness Checking** : vérification et ajustement du contenu pour éviter des “fabrications non justifiées”.
6. **Multi-Result** : récupération de plusieurs réponses pour un même prompt (OpenAI, Azure, Hugging Face).

Ce notebook est une **synthèse** des exemples dispersés dans différents notebooks, présentés sous forme de cellules Markdown et Python.


In [1]:
# ============================
# Cellule : Installation & Imports
# ============================

# N'installez qu'une seule fois si nécessaire
%pip install -U semantic-kernel

# Imports de base
import os
import sys
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.functions import KernelArguments
from semantic_kernel.contents import ChatHistory

print("Imports et installation OK.")


Note: you may need to restart the kernel to use updated packages.
Imports et installation OK.


## 3) Bloc Markdown – Configuration du Kernel & .env

```markdown
### Configuration du Kernel

Pour exécuter les exemples, on suppose que vous avez un fichier `.env` comportant vos clés d'API OpenAI / Azure OpenAI / Hugging Face.  
Exemple `.env` :

```
GLOBAL_LLM_SERVICE="OpenAI"        # ou AzureOpenAI, HuggingFace
OPENAI_API_KEY="sk-..."
OPENAI_CHAT_MODEL_ID="gpt-3.5-turbo"
...
```

Le `Kernel` lira ces informations pour décider quel connecteur LLM utiliser.  
Ensuite, nous ajouterons nos services (`OpenAIChatCompletion`, `AzureChatCompletion`, etc.) selon la variable `GLOBAL_LLM_SERVICE`.

In [2]:
# ============================
# Cellule : Initialisation du Kernel
# ============================

from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion
)

# On charge le .env
load_dotenv()
global_llm_service = os.getenv("GLOBAL_LLM_SERVICE", "AzureOpenAI")

# Initialisation du Kernel
kernel = Kernel()

if global_llm_service.lower() == "openai":
    # Ajout du service OpenAI
    kernel.add_service(
        OpenAIChatCompletion(service_id="chatOpenAI"),
    )
    print("Service OpenAI configuré.")
elif global_llm_service.lower() == "huggingface":
    # Ajout du service HuggingFace
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion
    kernel.add_service(
        HuggingFaceTextCompletion(
            service_id="chatHF",
            ai_model_id="distilgpt2", # ex. pour text-generation
            task="text-generation"
        ),
    )
    print("Service Hugging Face configuré.")
else:
    # Par défaut : Azure OpenAI
    kernel.add_service(
        AzureChatCompletion(service_id="chatAzure"),
    )
    print("Service Azure OpenAI configuré.")

Service OpenAI configuré.


### Chat Basique avec KernelArguments

L'idée : on crée une fonction de chat qui prend :
- l’historique de conversation (objet `ChatHistory`)
- un `user_input`
et on stocke le tout dans un `KernelArguments`.  

Cela permet d'alimenter un prompt (via un template) qui contient la variable `history` et `user_input`.

In [3]:
# ============================
# Cellule : Extrait d'un Chat Minimal
# ============================

# Exemple de prompt
chat_prompt = """
{{$history}}
User: {{$user_input}}
ChatBot:
"""

# Création d'une fonction sémantique "chat"
from semantic_kernel.prompt_template import PromptTemplateConfig, InputVariable

pt_config = PromptTemplateConfig(
    template=chat_prompt,
    name="chatFunction",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="User's message"),
        InputVariable(name="history", description="Conversation history"),
    ]
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="myChatPlugin",
    prompt_template_config=pt_config
)

chat_history = ChatHistory()
chat_history.add_system_message("Vous êtes un chatbot utile spécialisé en recommandations de livres.")

async def chat_kernel(input_text: str):
    print(f"[Utilisateur] : {input_text}")
    response = await kernel.invoke(
        chat_function,
        KernelArguments(user_input=input_text, history=chat_history)
    )
    print(f"[ChatBot] : {response}")
    # Mise à jour de l'historique
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(response))


# Test
await chat_kernel("Salut, peux-tu me conseiller un livre sur la philosophie antique ?")
await chat_kernel("Merci, tu peux détailler un peu plus la période concernée ?")

[Utilisateur] : Salut, peux-tu me conseiller un livre sur la philosophie antique ?
[ChatBot] : Salut ! Je te recommande "La République" de Platon. Ce dialogue explore la justice, la politique et la nature de l'âme, et il est fondamental pour comprendre la philosophie antique. Si tu cherches quelque chose de plus orienté vers les stoïciens, " Méditations" de Marc Aurèle est également une excellente lecture. Les deux ouvrages offrent des perspectives profondes sur la vie et la morale. Tu cherchais un genre particulier ou un auteur en particulier ?
[Utilisateur] : Merci, tu peux détailler un peu plus la période concernée ?
[ChatBot] : Bien sûr ! La philosophie antique s'étend principalement de la période présocratique (6e siècle av. J.-C.) jusqu'à la fin de l'Antiquité, vers le 6e siècle apr. J.-C. Cette époque englobe plusieurs courants de pensée et écoles :

1. **Les Présocratiques** (6e-5e siècle av. J.-C.) : Ce sont les premiers penseurs grecs comme Thalès, Anaximène et Héraclite, qui

### Planners : Orchestration Dynamique

**SequentialPlanner** : Génère un plan sous forme de liste d’étapes (XML), chaque étape étant une fonction existante.  
**FunctionCallingStepwisePlanner** : S'appuie sur OpenAI function-calling pour exécuter “pas à pas” (ReAct, MRKL).

**Exemple d'usage** :
1. Le *user* fournit un `goal`.
2. Le planner trouve les fonctions (plugins sémantiques ou natifs) permettant d'accomplir ce but.
3. Il exécute les étapes, éventuellement enchaînant *function calls*.

Ci-dessous, un extrait minimal de code pour un `SequentialPlanner`.

In [4]:
# ============================
# Cellule : Sequential Planner
# ============================

from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel, service_id="chatOpenAI")  # adapter selon ton service
user_goal = """
Demain c'est la Saint-Valentin. Je veux composer un poème
dans le style de Shakespeare, en français, puis convertir
le texte en majuscules.
"""

seq_plan = await planner.create_plan(user_goal)
print("Plan généré par le SequentialPlanner :")
for idx, step in enumerate(seq_plan._steps):
    print(f"Étape {idx+1} => {step.description} // {step.metadata.fully_qualified_name}")

# Exécuter le plan
execution_result = await seq_plan.invoke(kernel)
print("\n=== Résultat du plan ===")
print(execution_result)

Plan généré par le SequentialPlanner :
Étape 1 => None // myChatPlugin-chat
Étape 2 => None // myChatPlugin-chat

=== Résultat du plan ===
DANS LES JARDINS OÙ L’AUBE S’ÉVEILLE,  
LES FLEURS DANSENT SOUS LE SOUFFLE LÉGER,  
LE CIEL, PEINT D’OR, ÉTEND SA MERVEILLE,  
ET LE VENT MURMURE DES VERS ENFLAMMÉS.  

Ô TENDRE AMOUR, ÉTOILE DE MES NUITS,  
DE TON REGARD, JE SUIS ÉPERDUMENT ÉPRIS,  
COMME L'ABEILLE AU DOUX NECTAR, JE SUIS,  
EN TON SOURIRE, JE TROUVE MON ABRI.  

LES OMBRES DU TEMPS, TELLES DES VAGUES FUGACES,  
EMPORTENT LES RÊVES, MAIS JAMAIS TON IMAGE,  
CAR DANS L'ÉCRIN DE MON CŒUR, TU FAIS TA PLACE,  
ÉTERNITÉ, À TRAVERS MILLE ÂGES.  

QUANT AUX PLEURS DE L'ABSENCE, JE FAIS RIPOSTE,  
AMOUR, SOIS MON PHARE, DANS LA BRUME QUI S'IMPOSE.  
AVEC DES RIMES, À JAMAIS JE T’AUGUSTE,  
UNIS POUR L'ÉTERNITÉ, TEL UN VERS QUI EXPLOSE.  


### Mémoire & Embeddings

**SemanticTextMemory** permet de stocker des textes (avec un embedding) dans un store :
- `VolatileMemoryStore` (en mémoire)
- ou connecteurs vers Pinecone, Azure Cognitive Search, Qdrant, etc.

On peut ensuite effectuer des requêtes sémantiques :  
`await memory.search("MaCollection", "Quelle est mon budget pour 2024 ?")`

In [5]:
# ============================
# Cellule : Extrait Mémoire
# ============================

from semantic_kernel.memory import SemanticTextMemory, VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding

# Embedding service (ex : openai text-embedding-ada)
embedding_service = OpenAITextEmbedding(
    service_id="embeddingService",
    ai_model_id="text-embedding-3-small"
)

# Volatile in-memory store
store = VolatileMemoryStore()
memory = SemanticTextMemory(store, embeddings_generator=embedding_service)

# Ajouter ce plugin au kernel
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")

# Stocker quelques infos
collection_name = "testCollection"
await memory.save_information(collection=collection_name, id="info1", text="Budget 2024 = 100k€")
await memory.save_information(collection=collection_name, id="info2", text="Budget 2023 = 70k€")

# Requête
results = await memory.search(collection_name, "Quel est mon budget pour 2024 ?", limit=1)
print("Réponse potentielle : ", results[0].text)

Réponse potentielle :  Budget 2024 = 100k€


### Hugging Face Intégration

Semantic Kernel peut se connecter à Hugging Face localement ou via API.  
Exemple :  
```python
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

hf_service = HuggingFaceTextCompletion(
    service_id="textHF", ai_model_id="distilgpt2", task="text-generation"
)
kernel.add_service(hf_service)
```

Ensuite, on peut enregistrer une fonction sémantique ou invoquer directement `hf_service.get_text_contents(...)`.

### Groundedness Checking

Pour éviter les “hallucinations” d’un résumé :  
1. Extrait la liste d'entités du résumé.  
2. Vérifie la correspondance de chaque entité avec le texte source (référence).  
3. Retire ou corrige les entités non-fondées.

Cela se fait via un plugin “GroundingPlugin” (ex. ExtraitEntities, ReferenceCheckEntities, ExciseEntities).

In [8]:
# ============================
# Cellule : Groundedness Checking
# ============================

# Suppose qu'on a un "grounding_text" = un texte source
grounding_text = """
Votre budget 2024 est de 100k euros.
Vous vivez à Genève.
Vous avez investi 50k en actions.
"""

# Suppose qu'on a un résumé "faux"
summary_text = """
Mon budget 2024 est de 200k euros.
J'habite à Milan.
"""

plugins_directory = "./prompt_template_samples/"

# On appelle un plugin (hypothétique) "GroundingPlugin" comportant 3 fonctions
#   - "ExtractEntities"
#   - "ReferenceCheckEntities"
#   - "ExciseEntities"
grounding_plugin = kernel.add_plugin(parent_directory= plugins_directory, plugin_name="GroundingPlugin")

extract_entities = grounding_plugin["ExtractEntities"]
check_entities = grounding_plugin["ReferenceCheckEntities"]
excise_entities = grounding_plugin["ExciseEntities"]

# 1) Extraire entités
ext_result = await kernel.invoke(extract_entities, input=summary_text)
print("Entités détectées:", ext_result)

# 2) Vérifier correspondance
check_result = await kernel.invoke(check_entities, input=ext_result.value, reference_context=grounding_text)
print("Entités non-fondées:", check_result)

# 3) Retirer entités non-fondées du summary
excision = await kernel.invoke(excise_entities, input=summary_text, ungrounded_entities=check_result.value)
print("Summary corrigé:", excision)

Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments
Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments
Variable `Symbols.VAR_PREFIX: example_entities` not found in the KernelArguments
Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments
Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments
Variable `Symbols.VAR_PREFIX: topic` not found in the KernelArguments


Entités détectées: <entities>
- budget: Refers to the financial plan for the year 2024, indicating the amount of money allocated for spending.
- euros: The currency used in the budget, representing the monetary unit of the European Union.
- Milan: A city in Italy where the speaker resides.
</entities>
Entités non-fondées: <ungrounded_entities>
- Milan
</ungrounded_entities>
Summary corrigé: Mon budget 2024 est de 200k euros.


### Multi-Result

OpenAI (ou Azure) peut renvoyer plusieurs complétions pour un même prompt.  
On paramètre `number_of_responses=3` dans les settings.  
Ensuite, `get_text_contents(...)` ou `get_chat_message_contents(...)` renvoie un *tableau* de résultats.

In [12]:
# ============================
# Cellule : Multi-Result
# ============================

from semantic_kernel.connectors.ai.open_ai import OpenAITextPromptExecutionSettings

settings = OpenAITextPromptExecutionSettings(
    extension_data={
        "max_tokens": 60,
        "temperature": 0.7,
        "number_of_responses": 3
    }
)

from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion
text_service = OpenAITextCompletion(service_id="multiResult", ai_model_id="gpt-3.5-turbo-instruct")

prompt = "Donne-moi une brève blague sur les chats :"

# get_text_contents() => liste de réponses
responses = await text_service.get_text_contents(prompt, settings=settings)

for i, r in enumerate(responses):
    print(f"Réponse n°{i+1}:\n{r}\n")

Réponse n°1:


Pourquoi les chats n'aiment-ils pas l'eau ?

Parce que dans leur langue, "miaou" signifie "aïe, ça mouille !"

Réponse n°2:


Pourquoi les chats n'aiment pas les jeux de société ? Parce qu'ils préfèrent jouer avec les souris !

Réponse n°3:


Pourquoi les chats n'aiment-ils pas jouer au poker ? Parce qu'ils préfèrent garder leurs griffes pour le grattage des cartes !



## Conclusion

Nous avons vu :
1. Un **Chat** basique avec `KernelArguments`.
2. Les **Planners** (Sequential, Stepwise) pour orchestrer dynamiquement des steps.
3. La **Mémoire** & embeddings pour stocker/rechercher des informations sémantiques.
4. L’**intégration Hugging Face** pour exécuter localement des modèles open-source.
5. Un **Groundedness Checking** minimal pour éviter les hallucinations.
6. La gestion de **plusieurs réponses** (Multi-Result) avec un seul appel.

Ces fonctionnalités permettent de créer des scénarios complexes : chat évolué, question-answering avec mémoire persistante, planification automatique, usage local ou cloud, etc. Bonne exploration !